
# <center><font color= #F08080 > <b>CU34_Predicción de demanda de servicios</font></center>

#  <font color='green'>IV. Model development</font>

En este anexo se incluye el código utilizado durante el desarrollo 
de los modelos incluidos en el caso de uso.

In [1]:
Sys.setlocale(category = "LC_ALL", locale = "es_ES.UTF-8")

[1] "es_ES.UTF-8/es_ES.UTF-8/es_ES.UTF-8/C/es_ES.UTF-8/C"

## Paquetes



In [2]:
library(readr)
library(dplyr)
library(nnet)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Datos

In [3]:
datos <- read_rds("output_cluster.rds")


## Entrenamiento red

In [4]:
train <- sample(1:nrow(datos), 0.8*nrow(datos))
test <- -train

dfmod <- datos |> 
  select(cluster, Futbol:densidad_hab_km2) |> 
  mutate(cluster = factor(cluster)) |> 
  mutate(prec = if_else(prec < 0, 0, prec))
# Y <- datos |> 
#   transmute(cluster = as.character(cluster)) 

modelo <- nnet(cluster ~ ., size = 10, data = dfmod, subset = train,
               decay = 1, maxit = 200)

table(dfmod$cluster[-train], predict(modelo, dfmod[-train,], type = "class"))
table(dfmod$cluster[train], predict(modelo, dfmod[train,], type = "class"))

write_rds(modelo, "modelo_nnet.rds")

# weights:  370
initial  value 583571.630480 
iter  10 value 444761.909417
iter  20 value 435449.379991
iter  30 value 432242.463783
iter  40 value 431705.933202
iter  50 value 430680.186050
iter  60 value 426990.677363
iter  70 value 419497.102158
iter  80 value 407907.663764
iter  90 value 405058.294819
iter 100 value 403327.992297
iter 110 value 401786.358009
iter 120 value 400062.993717
iter 130 value 396265.793668
iter 140 value 388837.903810
iter 150 value 385109.716258
iter 160 value 382986.488711
iter 170 value 381765.170382
iter 180 value 379636.360605
iter 190 value 373280.734847
iter 200 value 370875.116692
final  value 370875.116692 
stopped after 200 iterations


    
        3    4    6    9
  1    15 3997  302  669
  2     0   32    0 2135
  3  4116  171 1042 2086
  4   265 4187 1231 1593
  5  1162 1477 1335 1340
  6   992 2637 8840  346
  7  2169  571  475 1741
  8     0 3634  127    7
  9  2068  334  454 2291
  10    0  108    0  624

    
         3     4     5     6     9
  1     70 16160     0  1154  2774
  2      0    98     0     0  8303
  3  16195   704     1  4132  8313
  4   1018 17066     0  4997  6161
  5   4737  5941     0  5489  5367
  6   3977 10454     0 35296  1390
  7   8475  2295     0  2039  6834
  8      0 14555     0   468    47
  9   8202  1343     0  1743  9349
  10     0   461     0     0  2681

## Escenario

In [5]:
set.seed(1)
escenario <- dfmod |> 
  select(-cluster) |> 
  slice_sample(n = 100)

write_csv(escenario, "ESCENARIO_SERVICIOS.csv")

## Predicción

In [6]:

## Tabla con el detalle:
pp <- predict(modelo, escenario) |> round(2)

## Gráfico de barras:
ppclass <- predict(modelo, escenario, type = "class")
ppclass

[1] "9" "6" "9" "6" "3" "4" "3" "4" "6" "3" "6" "6" "3" "4" "3" "6" "4" "6"
 [19] "3" "6" "4" "4" "4" "9" "4" "9" "4" "9" "6" "4" "6" "3" "6" "4" "6" "4"
 [37] "3" "4" "4" "6" "4" "9" "6" "3" "3" "6" "4" "6" "3" "6" "9" "4" "4" "3"
 [55] "4" "6" "4" "6" "4" "9" "6" "3" "6" "4" "9" "3" "4" "4" "4" "9" "4" "6"
 [73] "9" "3" "4" "4" "4" "4" "6" "4" "9" "4" "4" "9" "6" "3" "9" "3" "3" "3"
 [91] "6" "3" "3" "9" "4" "9" "6" "9" "3" "9"